This script is built around the custom FilterClass code I built, available in FilterClass.py

In [1]:
import FilterClass as fltr
from datetime import datetime
import xarray as xr
import gcpy
import copy

First initialize the MapFilter Class, and set MaskToNaNs to true to set all ocean values to NaN instead of 0

In [2]:
Filterer = fltr.MapFilter
Filterer.__init__(Filterer, MaskToNaNs=True)

RangeToFilter is the range over which to run the iteration script

In [3]:
RangeToFilter = ["20190804", "20191231"]
RangeDatetimes = [datetime.strptime(date, '%Y%m%d') for date in RangeToFilter]
RangeDatetimes

[datetime.datetime(2019, 8, 4, 0, 0), datetime.datetime(2019, 12, 31, 0, 0)]

Targeted_Species specified which species we're examining, and FileForOutput is the directory where all output will be saved

In [4]:
Targeted_Species = "Ethane"
DirForOutput = "Filtered_CrIS_Outputs/{}".format(Targeted_Species)
#DirForOutput = "/scratch.global/milletd/brewe222/data/Filtered_CrIS_Outputs/{}".format(Targeted_Species)

Window_Width will set the width of the window over which we will be averaging.

In [5]:
Window_Width = 2

These lines establish the Custom-built L2L regridder. The regridder is customized here to the in- and out-resolutions, and over the customized extent. This is also where the regridding method used is specified. By default, it is "bilinear".

In [6]:
llres_in="0.5x0.625"
llres_out="2x2.5"
extent_out=[-180, 180, -90, 90]

regridder = Filterer.custom_L2L_regridder(
    llres_in=llres_in, 
    llres_out= llres_out)

This loop is where the magic happens:

In [7]:
for day in fltr.daily_iterator(RangeDatetimes[0], RangeDatetimes[1]):
    print("  ")
    print(f"Processing {day.strftime('%Y-%m-%d')}")
    Averaged_Data = Filterer.get_column(
        Filterer, 
        longspeciesname=Targeted_Species, 
        date_targeted=day,
        window_width=Window_Width
    )
    
    Masked_Data = copy.deepcopy(Averaged_Data)
    Masked_Data['Column'] = Filterer.run_Landfilter(Filterer, column_array=Averaged_Data['Column'])
    
    Regridded_Data = Filterer.Regrid_CrIS(
        Filterer,
        regridder=regridder,
        dataset_to_regrid=Masked_Data,
        targetdatetime=day,
        llres_out=llres_out,
        extent=extent_out
    )
    
    print_date = day.strftime('%Y%m%d')
    save_name=f'{DirForOutput}/Regrid_{llres_out}_{print_date}_{Targeted_Species}_Column.nc'
    print(f'Saving {save_name}')
    print("  ")
    Regridded_Data.to_netcdf(save_name)

  
Processing 2019-08-04
Multiple files supplied. Columns represent 5 days worth of data
Saving Filtered_CrIS_Outputs/Ethane/Regrid_2x2.5_20190804_Ethane_Column.nc
  
